In [6]:
import cvxpy as cp

# Number of cities
num_cities = 8

# Costs per day (in USD)
cost_weekend = [60, 20, 40, 25, 25, 20, 30, 30]  # Fri/Sat
cost_transition = [25, 15, 25, 20, 30, 20, 30, 25]  # Sun/Thu
cost_weekday = [25, 15, 20, 20, 20, 20, 30, 25]  # Mon/Tue/Wed

# Travel costs (matrix in USD)
travel_cost = [
    [0, 40, 65, 20, 50, 30, 50, 60],
    [40, 0, 105, 40, 130, 70, 165, 130],
    [65, 105, 0, 100, 60, 25, 30, 40],
    [20, 40, 100, 0, 86, 20, 66, 75],
    [50, 130, 60, 86, 0, 100, 80, 20],
    [30, 70, 25, 20, 100, 0, 90, 130],
    [50, 165, 30, 66, 80, 90, 0, 40],
    [60, 130, 40, 75, 20, 130, 40, 0]
]

# Decision variables
x = cp.Variable(num_cities, integer=True)  # Total days spent in each city
y_weekday = cp.Variable(num_cities, integer=True)
y_weekend = cp.Variable(num_cities, integer=True)
y_transition = cp.Variable(num_cities, integer=True)
t = cp.Variable((num_cities, num_cities), boolean=True)  # Travel binary variables

# Total budget and days
budget = 1500
max_days = 14

# Objective: Minimize total cost
total_cost = (
    sum(cost_weekend[i] * y_weekend[i] for i in range(num_cities)) +
    sum(cost_weekday[i] * y_weekday[i] for i in range(num_cities)) +
    sum(cost_transition[i] * y_transition[i] for i in range(num_cities)) +
    sum(
        travel_cost[i][j] * t[i, j] 
        for i in range(num_cities) 
        for j in range(num_cities)
    )
)

# Constraints
constraints = [
    x >= 0,
    y_weekday >= 0,
    y_weekend >= 0,
    y_transition >= 0,
    x <= 3,  # No more than 3 days in a single city
    sum(x) == max_days,  # Total days equal to trip duration
    sum(
        cost_weekend[i] * y_weekend[i] +
        cost_weekday[i] * y_weekday[i] +
        cost_transition[i] * y_transition[i] +
        sum(travel_cost[i][j] * t[i, j] for j in range(num_cities))
        for i in range(num_cities)
    ) <= budget
]

#add constraints for day of week
constraints.append(sum(y_weekday) == 6)
constraints.append(sum(y_weekend) == 4)
constraints.append(sum(y_transition) == 4)

#add constraints for number of consecutive days per city
for i in range(num_cities):
    constraints.append(y_weekday[i] <= 3)
    constraints.append(y_weekend[i] <= 2)
    constraints.append(y_transition[i] <= 2)

# Add individual constraints for day breakdown
for i in range(num_cities):
    constraints.append(y_weekday[i] + y_weekend[i] + y_transition[i] == x[i])

# Solver
problem = cp.Problem(cp.Minimize(total_cost), constraints)
problem.solve(solver=cp.GUROBI)

# Results
print("Optimal Cost:", problem.value)
print("Days in each city:", [round(val) for val in x.value])
print("Weekdays in each city:", [round(val) for val in y_weekday.value])
print("Weekends in each city:", [round(val) for val in y_weekend.value])
print("Transits in each city:", [round(val) for val in y_transition.value])

Optimal Cost: 275.0
Days in each city: [0, 3, 2, 3, 3, 3, 0, 0]
Weekdays in each city: [0, 2, 2, 0, 2, 0, 0, 0]
Weekends in each city: [0, 0, 0, 1, 1, 2, 0, 0]
Transits in each city: [0, 1, 0, 2, 0, 1, 0, 0]
